# Prediction of teams that will reach the Playoffs

## Data Import

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
! pip install tabulate
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency

players = pd.read_csv('basketballPlayoffs/players.csv', delimiter=",")
coaches = pd.read_csv('basketballPlayoffs/coaches.csv', delimiter=",")
teams = pd.read_csv('basketballPlayoffs/teams.csv', delimiter=",")
players_teams = pd.read_csv('basketballPlayoffs/players_teams.csv', delimiter=",")
teams_post = pd.read_csv('basketballPlayoffs/teams_post.csv', delimiter=",")
series_post = pd.read_csv('basketballPlayoffs/series_post.csv', delimiter=",")
awards_players = pd.read_csv('basketballPlayoffs/awards_players.csv', delimiter=",")
awards_coaches = pd.read_csv('basketballPlayoffs/awards_coaches.csv', delimiter=",")

print(players.head())
print(coaches)
print(teams)
print(players_teams)
print(teams_post)
print(series_post)
print(awards_players)
print(awards_coaches)

## Data cleaning

Check for missing values, outliers, and inconsistencies in the data. Clean and preprocess the data to ensure it's ready for analysis.

### Teams dataset

In [ ]:
print(teams.isna().sum())
print(teams.head())

### Players Teams dataset

In [ ]:
print(players_teams.isna().sum())
print(players_teams.head())

### Players dataset

In [ ]:
print(players.isna().sum())
print(players.head())

### Coaches dataset

In [ ]:
print(coaches.isna().sum())
print(coaches.head())

### Awards Players dataset

In [ ]:
print(awards_players.isna().sum())
print(awards_players.head())

### Awards Coaches dataset

In [ ]:
print(awards_coaches.isna().sum())
print(awards_coaches.head())

## Exploratory Data Analysis (EDA)

Conduct EDA to gain insights into the data. Visualize distributions, correlations, and patterns. This step will help you understand the relationships between different features and the target variable.

### Teams metrics

In [ ]:
teams.head()

teams.isnull().sum()

In [ ]:
for column in teams.columns:
    unique_values = teams[column].unique()
    print(f"Number of different values in the {column} column are:", len(unique_values))
    print("------------")

In [ ]:
columns = []
value_counts = []

for column in teams.columns:
    unique_values = teams[column].nunique()
    columns.append(column)
    value_counts.append(unique_values)

plt.figure(figsize=(10, 10))
plt.barh(columns, value_counts, color='skyblue')
plt.xlabel('Number of Unique Values')
plt.ylabel('Columns')
plt.title('Number of Unique Values in Each Column')
plt.show()

In [ ]:
print(teams.dtypes)

In [ ]:
teams_numeric = teams.copy()

for column in teams_numeric.columns:
    if teams_numeric[column].dtype == 'object':
        teams_numeric[column] = teams_numeric[column].astype('category').cat.codes

teams_numeric.describe()

In [ ]:
def correlation_matrix(dataframe):
    corr_matrix = dataframe.corr()

    target_correlation = corr_matrix['playoff']

    plt.figure(figsize=(30, 20))

    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    sns.heatmap(corr_matrix, mask=mask, annot=True, annot_kws={"size": 8}, cmap='coolwarm', linewidths=0.5, fmt=".2f")

    plt.title('Correlation Matrix', fontsize=16)
    plt.show()

    dict = {}

    for feature, correlation in target_correlation.items():
        print(f"Correlation between target and {feature}: {correlation}")
        dict[feature] = correlation

    return dict

In [ ]:
correlation_matrix(teams_numeric)

In [ ]:
def chi_square(dataset, target): 
    for feature in dataset.columns:
        if feature != target:
            contingency_table = pd.crosstab(dataset[feature], dataset[target])

            # check if any category has no data
            if contingency_table.shape[0] == 0 or contingency_table.shape[1] == 0:
                print(f"No data for {feature} and {target}")
                continue
            
            chi2, p, observed, expected = chi2_contingency(contingency_table)
            
            # Step 4: Print or store the results
            print(f"Chi-square test for {feature} and {target}:")
            print(f"Chi-square value: {chi2}")
            print(f"P-value: {p}")
            print("")

In [ ]:
chi_square(teams, 'playoff')

The p-value indicates the probability of observing a relationship as extreme as the one in our sample data, assuming that there is no actual relationship in the population.

### Players metrics

In [ ]:
print(players['playerID'].nunique()) 

print(players.head())

players.isnull().sum()

#### Erased columns and why

In [ ]:
# Dropped columns: 'collegeOther', 'deathDate', 'firstseason', 'lastseason'

# Count the rows where 'firstseason' is not equal to 0
non_zero_firstseason_count = len(players[players['firstseason'] != 0])

# Count the rows where 'firstseason' is not equal to 0
non_zero_lastseason_count = len(players[players['lastseason'] != 0])

# Count the rows where 'deathDate' is not equal to "0000-00-00"
players['deathDate'] = players['deathDate'].str.strip()
non_empty_deathDate_count = len(players[players['deathDate'] != "0000-00-00"])

# Count the rows where 'collegeOther' is not equal to ""
non_nan_collegeOther_count = players['collegeOther'].notna().sum()

print("Number of rows with 'firstseason' different from 0:", non_zero_firstseason_count)
print("Number of rows with 'lastseason' different from 0:", non_zero_lastseason_count)
print("Number of rows with 'collegeOther' different from "":", non_nan_collegeOther_count)
print("Number of rows with 'deathDate' different from '0000-00-00':", non_empty_deathDate_count)

#### Players heights comparison

In [ ]:
# Convert height from inches to centimeters
players['height_cm'] = players['height'] * 2.54  # 1 inch = 2.54 cm

# Define height categories in centimeters
height_categories = ['< 160.0 cm', '160.0 - 170.0 cm', '170.0 - 180.0 cm', '180.0 - 190.0 cm', '190.0 - 200.0 cm', '> 200.0 cm']

# Define the height ranges for each category
height_ranges = [(0, 160.0), (160.0, 170.0), (170.0, 180.0), (180.0, 190.0), (190.0, 200.0), (200.0, float('inf'))]

# Create a new column in the dataset to store the height category for each player
players['height_category'] = pd.cut(players['height_cm'], bins=[r[0] for r in height_ranges] + [float('inf')], labels=height_categories)

# Count the number of players in each height category
height_category_counts = players['height_category'].value_counts().reindex(height_categories, fill_value=0)

# Create a bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=height_category_counts.index, y=height_category_counts.values, palette='Set2')

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

# Set plot title and labels
plt.title('Player Count in Height Categories')
plt.xlabel('Height Category')
plt.ylabel('Count')

# Show the plot
plt.show()

#### Number of players in each position

In [ ]:
# Create a countplot for player positions
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
ax = sns.countplot(data=players, x='pos', order=players['pos'].value_counts().index, palette='Set2')

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

# Set plot title and labels
plt.title('Number of Players in Each Position')
plt.xlabel('Position')
plt.ylabel('Count')

# Rotate x-axis labels for better readability (optional)
plt.xticks(rotation=45)

empty_pos_count = players['pos'].isnull().sum()
print("Number of rows with empty 'pos':", empty_pos_count)

# Show the plot
plt.show()

#### Top 10 Colleges

In [ ]:
# Get the top 10 colleges with the most players
top_10_colleges = players['college'].value_counts().iloc[:10]

# Create a countplot for the top 10 colleges
plt.figure(figsize=(12, 6))  # Adjust the figure size as needed
ax = sns.countplot(data=players, x='college', order=top_10_colleges.index, palette='Set2')

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

# Set plot title and labels
plt.title('Top 10 Colleges with the Most Players')
plt.xlabel('College')
plt.ylabel('Count')

# Rotate x-axis labels for better readability (optional)
plt.xticks(rotation=90)

# Show the plot
plt.show()

#### Correlation Matrix between numeric columns

In [ ]:
# Select the columns for correlation analysis
numeric_columns = ["firstseason", "lastseason", "height", "weight"]

# Create a subset of the dataset with only the numeric columns
subset = players[numeric_columns]

# Calculate the correlation matrix
corr_matrix = subset.corr()

# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

### Teams Post metrics

In [ ]:
print(teams_post.head())

teams_post.isnull().sum()

In [ ]:
# Calculate win-loss ratios
teams_post['Win-Loss Ratio'] = teams_post['W'] / (teams_post['W'] + teams_post['L'])

# Create a horizontal bar chart
plt.figure(figsize=(12, 8))
plt.barh(teams_post['tmID'], teams_post['Win-Loss Ratio'], color='skyblue')

# Add labels and title
plt.xlabel('Win-Loss Ratio')
plt.ylabel('Team ID (tmID)')
plt.title('Win-Loss Ratios for Teams on  Post-Season (based on tmID)')

# Show the chart
plt.show()

### Series Post metrics

In [ ]:
# Group the data by year and count the number of wins for each team in each year
team_wins_by_year = series_post.groupby(['year', 'tmIDWinner'])['W'].count().unstack(fill_value=0)

# Create a stacked bar chart
plt.figure(figsize=(12, 8))
team_wins_by_year.plot(kind='bar', stacked=True, colormap='Set3')
plt.xlabel('Year')
plt.ylabel('Number of Wins')
plt.title('Teams That Won In The Playoffs Each Year')

# Show the chart
plt.legend(title='Team', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

#### Teams that won and lost each year

In [ ]:
# Filter for the "F" (Finals) round
finals_data = series_post[series_post['round'] == 'F']

# Create a DataFrame with the winning and losing teams for each year
finals_results = finals_data[['year', 'tmIDWinner', 'tmIDLoser']]

# Convert the DataFrame to a prettily formatted table
table = tabulate(finals_results, headers='keys', tablefmt='fancy_grid', showindex=False)

# Display the formatted table
print(table)

In [ ]:
# Filter for the "F" (Finals) round
finals_data = series_post[series_post['round'] == 'F']

# Count how many times each team has appeared in the Finals as either a winner or a loser
team_appearances = pd.concat([finals_data['tmIDWinner'], finals_data['tmIDLoser']]).value_counts()

# Create a horizontal bar chart
plt.figure(figsize=(12, 6))
team_appearances.plot(kind='barh', color='skyblue')

# Add labels and title
plt.ylabel('Team (tmID)')
plt.xlabel('Number of Finals Appearances')
plt.title('Total Appearances of Each Team in the Finals')

# Show the chart
plt.show()

### Coaches metrics

In [ ]:
print(coaches['coachID'].nunique()) 

print(coaches.head())

coaches.isnull().sum()

In [ ]:
# Extract the wins and losses data from the "won" and "lost" columns
coach_wins = coaches['won']
coach_losses = coaches['lost']

# Create a scatter plot
import matplotlib.pyplot as plt

plt.scatter(coach_wins, coach_losses, alpha=0.5)
plt.xlabel('Wins')
plt.ylabel('Losses')
plt.title('Scatter Plot of Coach Wins vs. Losses')
plt.show()

## Feature Selection

Identify and select relevant features for your prediction model. Use techniques such as correlation analysis, recursive feature elimination, or feature importance from tree-based models.

### Teams dataset

In [ ]:
original_teams = teams.copy()

In [ ]:
# to delete: lgID, divID, seeded, tmORB, tmDRB, tmTRB, opptmORB, opptmDRB, opptmTRB, rank, firstRound, semis, finals
print(teams.isna().sum())
   
feature_selection_result = teams.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
feature_selection_result = feature_selection_result.drop(columns=['rank', 'firstRound', 'semis', 'finals'])
feature_selection_result = feature_selection_result.drop(columns=['min', 'o_oreb', 'o_dreb', 'd_oreb', 'd_dreb', 'name', 'franchID'])

feature_selection_result.head()

In [ ]:
feature_selection_result.to_csv('filtered/feature_selection_dataset.csv', index=False)

## Feature Engineering

Create new features that might enhance the predictive power of your model. This could involve transforming existing features, creating interaction terms, or incorporating external data.

The feauture enginnering is done inside the `players.ipynb` file and the creation of the new dataset is done inside `create_final_team.csv` file.

### Shifting target variable

In [ ]:
def shift_target_variable(dataset):
    dataset.sort_values(by=['tmID', 'year'], inplace=True)

    dataset['playoffs'] = dataset.groupby('tmID')['playoff'].shift(-1)

    dataset.drop(columns=['playoff'], inplace=True)

    dataset.dropna(subset=['playoffs'], inplace=True)

    dataset.rename(columns={'playoffs': 'playoff'}, inplace=True)

    dataset.reset_index(drop=True, inplace=True)

    return dataset

#### Original dataset

In [ ]:
original_teams = shift_target_variable(original_teams)
original_teams.to_csv('filtered/original_teams.csv', index=False)

In [ ]:
original_teams_numeric = original_teams.copy()

for column in original_teams_numeric.columns:
    if original_teams_numeric[column].dtype == 'object':
        original_teams_numeric[column] = original_teams_numeric[column].astype('category').cat.codes

correlation_matrix(original_teams_numeric)

#### Feature Selection dataset

In [ ]:
feature_selection_result = shift_target_variable(feature_selection_result)

In [ ]:
feature_teams_numeric = feature_selection_result.copy()

for column in feature_teams_numeric.columns:
    if feature_teams_numeric[column].dtype == 'object':
        feature_teams_numeric[column] = feature_teams_numeric[column].astype('category').cat.codes

correlation_matrix(feature_teams_numeric)

#### Feature engineering dataset

In [ ]:
# TODO: verificar se pode ficar assim ou se adicionamos o código cá
feature_engineering_result = pd.read_csv('filtered/feature_engineering_dataset.csv', delimiter=",")

In [ ]:
eng1_teams_numeric = shift_target_variable(feature_engineering_result.copy())
eng1_teams_numeric.to_csv('filtered/eng1_teams_numeric.csv', index=False)

In [ ]:
eng1_teams_numeric = eng1_teams_numeric.copy()

for column in eng1_teams_numeric.columns:
    if eng1_teams_numeric[column].dtype == 'object':
        eng1_teams_numeric[column] = eng1_teams_numeric[column].astype('category').cat.codes

correlation_matrix(eng1_teams_numeric)

In [ ]:
feature_engineering_result2 = pd.read_csv('filtered/team2_before_shift.csv', delimiter=",")
feature_engineering_result2.drop(columns=['playoff'], inplace=True)
copy_fe1 = feature_engineering_result.copy()
copy_fe1.drop(columns=['powerRanking2'], inplace=True)

feature_engineering_result2 = pd.merge(feature_engineering_result2, copy_fe1, on=['tmID', 'year'])
feature_engineering_result2.to_csv('text.csv', index=False)
feature_engineering_result2 = shift_target_variable(feature_engineering_result2)
feature_engineering_result = shift_target_variable(feature_engineering_result)

#feature_engineering_result2.to_csv('text.csv', index=False)

feature_engineering_result2.head()

In [ ]:
eng2_teams_numeric = feature_engineering_result2.copy()

for column in eng2_teams_numeric.columns:
    if eng2_teams_numeric[column].dtype == 'object':
        eng2_teams_numeric[column] = eng2_teams_numeric[column].astype('category').cat.codes

correlation_matrix(eng2_teams_numeric)

In [ ]:
continuous = ['year', 'average_powerRanking', 'average_PER', 'average_postPowerRanking', 'average_postPER']

In [ ]:
eng2_teams_numeric.head()

## Classification

### Encoding

#### Original

In [ ]:
#tmID, confID, playoff, arena
# name, franchID, lgID, divID

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the columns 'teamID', 'franchID', 'confID', 'name', 'arena'
original_teams['tmID'] = label_encoder.fit_transform(original_teams['tmID'])
original_teams['confID'] = label_encoder.fit_transform(original_teams['confID'])
original_teams['arena'] = label_encoder.fit_transform(original_teams['arena'])
original_teams['name'] = label_encoder.fit_transform(original_teams['name'])
original_teams['franchID'] = label_encoder.fit_transform(original_teams['franchID'])
original_teams['lgID'] = label_encoder.fit_transform(original_teams['lgID'])
original_teams['divID'] = label_encoder.fit_transform(original_teams['divID'])
original_teams['firstRound'] = label_encoder.fit_transform(original_teams['firstRound'])
original_teams['semis'] = label_encoder.fit_transform(original_teams['semis'])
original_teams['finals'] = label_encoder.fit_transform(original_teams['finals'])

#### After Feature Selection

In [ ]:
feature_selection_result['tmID'] = label_encoder.fit_transform(feature_selection_result['tmID'])
feature_selection_result['confID'] = label_encoder.fit_transform(feature_selection_result['confID'])
feature_selection_result['arena'] = label_encoder.fit_transform(feature_selection_result['arena'])

#### After feature engineering

In [ ]:
# Apply label encoding to the columns 'teamID'
feature_engineering_result['tmID'] = label_encoder.fit_transform(feature_engineering_result['tmID'])

In [ ]:
feature_engineering_result2['tmID'] = label_encoder.fit_transform(feature_engineering_result2['tmID'])
feature_engineering_result2['confID'] = label_encoder.fit_transform(feature_engineering_result2['confID'])
""" feature_engineering_result2['arena'] = label_encoder.fit_transform(feature_engineering_result2['arena'])
feature_engineering_result2['firstRound'] = label_encoder.fit_transform(feature_engineering_result2['firstRound'])
feature_engineering_result2['semis'] = label_encoder.fit_transform(feature_engineering_result2['semis'])
feature_engineering_result2['finals'] = label_encoder.fit_transform(feature_engineering_result2['finals']) """

### Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np

from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, scoring=None, train_sizes=np.linspace(0.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, scoring=scoring, train_sizes=train_sizes, n_jobs=-1
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()

    plt.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    plt.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    plt.plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    plt.plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )

    plt.legend(loc="best")
    return plt

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

def decision_tree_model(dataset, year):
    X = dataset.copy()
    y = dataset['playoff'].copy()
    X.drop(columns=['playoff'], inplace=True)

    X_train = dataset[dataset['year'] < year].drop(columns=['playoff'])
    y_train = dataset[dataset['year'] < year]['playoff']

    X_test = dataset[dataset['year'] == year].drop(columns=['playoff'])
    y_test = dataset[dataset['year'] == year]['playoff']

    # Define the hyperparameters and their possible values
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 2, 3, 4, 5, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Create a GridSearchCV object
    dt_classifier = DecisionTreeClassifier()
    grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Step 2: Extract the best parameters from Grid Search
    best_params = grid_search.best_params_

    # Step 3: Create a Decision Tree classifier with the best parameters
    best_model = DecisionTreeClassifier(**best_params)

    # Step 4: Perform Recursive Feature Elimination (RFE)
    rfe = RFE(estimator=best_model, n_features_to_select=1)
    X_rfe = rfe.fit_transform(X_train, y_train)

    # Step 5: Train the final Decision Tree model with the best parameters and selected features
    best_model.fit(X_rfe, y_train)

    # Step 6: Evaluate the model on the test set
    X_test_rfe = rfe.transform(X_test)
    y_pred = best_model.predict(X_test_rfe)

    title = "Learning Curves (Decision Tree)"  # Change the title accordingly
    scoring = "accuracy"  # Change to your preferred scoring metric

    # Assuming you have a function named plot_learning_curve
    plot_learning_curve(
        best_model,
        title,
        X_train,
        y_train,
        cv=5,  # Number of cross-validation folds
        scoring=scoring,
    )

    plt.show()

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    print(classification_report(y_test, y_pred, zero_division=1))

##### Original dataset

In [ ]:
decision_tree_model(original_teams, 9)

##### Feature Selection dataset

In [ ]:
decision_tree_model(feature_selection_result, 9)

##### Feature engineering dataset

In [ ]:
decision_tree_model(feature_engineering_result, 9)

In [ ]:
decision_tree_model(feature_engineering_result2, 9)

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

def random_forest_model(dataset, year):
    X = dataset.copy()
    y = dataset['playoff'].copy()
    X.drop(columns=['playoff'], inplace=True)

    X_train = dataset[dataset['year'] < year].drop(columns=['playoff'])
    y_train = dataset[dataset['year'] < year]['playoff']

    X_test = dataset[dataset['year'] == year].drop(columns=['playoff'])
    y_test = dataset[dataset['year'] == year]['playoff']

    # Define the hyperparameters and their possible values
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10, 20, 30],
        'min_samples_leaf': [1, 2, 5, 10], 
        'max_features': ['auto', 'sqrt', 'log2', None]
    }

    # Create a GridSearchCV object
    rf_classifier = RandomForestClassifier()
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Step 2: Extract the best parameters from Grid Search
    best_params = grid_search.best_params_

    # Step 3: Create a Random Forest classifier with the best parameters
    best_model = RandomForestClassifier(**best_params)

    # Step 4: Perform Recursive Feature Elimination (RFE)
    rfe = RFE(estimator=best_model, n_features_to_select=1)
    X_rfe = rfe.fit_transform(X_train, y_train)

    # Step 5: Train the final Random Forest model with the best parameters and selected features
    best_model.fit(X_rfe, y_train)

    # Step 6: Evaluate the model on the test set
    X_test_rfe = rfe.transform(X_test)
    y_pred = best_model.predict(X_test_rfe)

    title = "Learning Curves (Random Forest)"  # Change the title accordingly
    scoring = "accuracy"  # Change to your preferred scoring metric

    # Assuming you have a function named plot_learning_curve
    plot_learning_curve(
        best_model,
        title,
        X_train,
        y_train,
        cv=5,  # Number of cross-validation folds
        scoring=scoring,
    )

    plt.show()

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    print(classification_report(y_test, y_pred, zero_division=1))

##### Original dataset

In [ ]:
random_forest_model(original_teams, 9)

##### Feature Selection dataset

In [ ]:
random_forest_model(feature_selection_result, 9)

##### Feature engineering dataset

In [ ]:
random_forest_model(feature_engineering_result, 9)

In [ ]:
random_forest_model(feature_engineering_result2, 9)

In [ ]:
for i in range(4, 10): 
    print("Year: ", i)
    random_forest_model(feature_engineering_result2, i)

In [ ]:
#for i in range(1, 8): 
    

#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

def logistic_regression_model(dataset, year):
    X = dataset.copy()
    y = dataset['playoff'].copy()
    X.drop(columns=['playoff'], inplace=True)

    X_train = dataset[dataset['year'] < year].drop(columns=['playoff'])
    y_train = dataset[dataset['year'] < year]['playoff']

    X_test = dataset[dataset['year'] == year].drop(columns=['playoff'])
    y_test = dataset[dataset['year'] == year]['playoff']

    # Define the hyperparameters and their possible values
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'lbfgs', 'saga'],
        'max_iter': [1000, 10000]
    }

    # Create a GridSearchCV object
    lr_classifier = LogisticRegression()
    grid_search = GridSearchCV(estimator=lr_classifier, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Step 2: Extract the best parameters from Grid Search
    best_params = grid_search.best_params_

    # Step 3: Create a Logistic Regression classifier with the best parameters
    best_model = LogisticRegression(**best_params)

    # Step 4: Perform Recursive Feature Elimination (RFE)
    rfe = RFE(estimator=best_model, n_features_to_select=1)
    X_rfe = rfe.fit_transform(X_train, y_train)

    # Step 5: Train the final Logistic Regression model with the best parameters and selected features
    best_model.fit(X_rfe, y_train)

    # Step 6: Evaluate the model on the test set
    X_test_rfe = rfe.transform(X_test)
    y_pred = best_model.predict(X_test_rfe)

    title = "Learning Curves (Logistic Regression)"  # Change the title accordingly
    scoring = "accuracy"  # Change to your preferred scoring metric

    # Assuming you have a function named plot_learning_curve
    plot_learning_curve(
        best_model,
        title,
        X_train,
        y_train,
        cv=5,  # Number of cross-validation folds
        scoring=scoring,
    )

    plt.show()

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    print(classification_report(y_test, y_pred, zero_division=1))

##### Original dataset

In [ ]:
logistic_regression_model(original_teams, 9)

##### Feature Selection dataset

In [ ]:
logistic_regression_model(feature_selection_result, 9)

##### Feature Engineering dataset

In [ ]:
logistic_regression_model(feature_engineering_result, 9)

In [ ]:
logistic_regression_model(feature_engineering_result2, 9)

In [ ]:
for i in range(4, 10):
    print("Year: ", i)
    logistic_regression_model(feature_engineering_result2, i)

#### Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

def svm_model(dataset, year): 
    X = dataset.copy()
    y = dataset['playoff'].copy()
    X.drop(columns=['playoff'], inplace=True)

    X_train = dataset[dataset['year'] < year].drop(columns=['playoff'])
    y_train = dataset[dataset['year'] < year]['playoff']

    X_test = dataset[dataset['year'] == year].drop(columns=['playoff'])
    y_test = dataset[dataset['year'] == year]['playoff']

    # Scale the features using StandardScaler
    scaler = StandardScaler()
    #scaler = RobustScaler()
    #scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Define the hyperparameters and their possible values
    param_grid = {
        'C': [0.1, 1],
        'kernel': ['linear', 'rbf'],
        'gamma': ['auto', 'scale'], 
    }

    # Create a GridSearchCV object
    svc = SVC()
    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Step 2: Extract the best parameters from Grid Search
    best_params = grid_search.best_params_

    # Step 3: Create an SVM classifier with the best parameters
    best_model = SVC(**best_params)

    # Step 4: Perform Recursive Feature Elimination (RFE) with a linear SVM
    linear_svc = LinearSVC()
    rfe = RFE(estimator=linear_svc, n_features_to_select=10)
    X_rfe = rfe.fit_transform(X_train, y_train)

    # Step 5: Train the final SVM model with the best parameters and selected features
    best_model.fit(X_rfe, y_train)

    # Step 6: Evaluate the model on the test set
    X_test_rfe = rfe.transform(X_test)
    y_pred = best_model.predict(X_test_rfe)

    title = "Learning Curves (SVM)"  # Change the title accordingly
    scoring = "accuracy"  # Change to your preferred scoring metric

    # Assuming you have a function named plot_learning_curve
    plot_learning_curve(
        best_model,
        title,
        X_train,
        y_train,
        cv=5,  # Number of cross-validation folds
        scoring=scoring,
    )

    plt.show()

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    print(classification_report(y_test, y_pred, zero_division=1))

##### Original dataset

In [ ]:
svm_model(original_teams, 9)

##### Feature Selection dataset

In [ ]:
svm_model(feature_selection_result, 9)

##### Feature engineering dataset

In [ ]:
svm_model(feature_engineering_result, 9)

In [ ]:
svm_model(feature_engineering_result2, 9)

#### KNN dataset

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

def knn_model(dataset, year):
    X = dataset.copy()
    y = dataset['playoff'].copy()
    X.drop(columns=['playoff'], inplace=True)

    X_train = dataset[dataset['year'] < year].drop(columns=['playoff'])
    y_train = dataset[dataset['year'] < year]['playoff']

    X_test = dataset[dataset['year'] == year].drop(columns=['playoff'])
    y_test = dataset[dataset['year'] == year]['playoff']

    # Scale the features using StandardScaler
    scaler = StandardScaler()
    #scaler = RobustScaler()
    #scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Use SelectKBest for feature selection
    k_best_selector = SelectKBest(f_classif, k='all')  # You can adjust 'k' based on your preference
    X_train_selected = k_best_selector.fit_transform(X_train_scaled, y_train)

    # Define the hyperparameters and their possible values
    param_grid = {
        'n_neighbors': [1, 3, 5, 7, 10],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'p': [1, 2]
    }

    # Create a GridSearchCV object
    knn_classifier = KNeighborsClassifier()
    
    grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train_selected, y_train)

    # Extract the best parameters from Grid Search
    best_params = grid_search.best_params_

    # Create the final k-Nearest Neighbors classifier with the best parameters
    best_model = KNeighborsClassifier(**best_params)

    # Train the final model with the selected features
    best_model.fit(X_train_selected, y_train)

    # Evaluate the model on the scaled test set
    X_test_selected = k_best_selector.transform(X_test_scaled)
    y_pred = best_model.predict(X_test_selected)

    title = "Learning Curves (KNN)"  # Change the title accordingly
    scoring = "accuracy"  # Change to your preferred scoring metric

    plot_learning_curve(
        best_model,
        title,
        X_train_selected,
        y_train,
        cv=5,  # Number of cross-validation folds
        scoring=scoring,
    )

    plt.show()

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    print(classification_report(y_test, y_pred))

##### Original dataset

In [ ]:
knn_model(original_teams, 9)

##### Feature Selection dataset

In [ ]:
knn_model(feature_selection_result, 9)

##### Feature engineering dataset

In [ ]:
knn_model(feature_engineering_result, 9)

In [ ]:
knn_model(feature_engineering_result2, 9)

#### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

def gradient_boosting_model(dataset, year):
    X = dataset.copy()
    y = dataset['playoff'].copy()
    X.drop(columns=['playoff'], inplace=True)

    X_train = dataset[dataset['year'] < year].drop(columns=['playoff'])
    y_train = dataset[dataset['year'] < year]['playoff']

    X_test = dataset[dataset['year'] == year].drop(columns=['playoff'])
    y_test = dataset[dataset['year'] == year]['playoff']

    # Define the hyperparameters and their possible values
    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.8, 1.0],
    }

    # Create a GradientBoostingClassifier object
    gb_classifier = GradientBoostingClassifier()

    # Create an RFECV object
    rfecv = RFECV(estimator=gb_classifier, step=1, cv=5)  # You can adjust 'cv' based on your preference

    # Perform RFECV on the training data
    X_train_rfecv = rfecv.fit_transform(X_train, y_train)

    # Extract the optimal number of features
    optimal_num_features = rfecv.n_features_

    print(f"Optimal number of features: {optimal_num_features}")

    # Create a new GradientBoostingClassifier with the best parameters
    grid_search = GridSearchCV(estimator=gb_classifier, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train_rfecv, y_train)

    # Extract the best parameters from Grid Search
    best_params = grid_search.best_params_

    # Create the final Gradient Boosting classifier with the best parameters
    best_model = GradientBoostingClassifier(**best_params)

    # Train the final model with the optimal features
    best_model.fit(X_train_rfecv, y_train)

    # Evaluate the model on the test set
    X_test_rfecv = rfecv.transform(X_test)
    y_pred = best_model.predict(X_test_rfecv)

    title = "Learning Curves (Gradient Boosting)"  # Change the title accordingly
    scoring = "accuracy"  # Change to your preferred scoring metric

    # Assuming you have a function named plot_learning_curve
    plot_learning_curve(
        best_model,
        title,
        X_train_rfecv,
        y_train,
        cv=5,  # Number of cross-validation folds
        scoring=scoring,
    )

    plt.show()

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    print(classification_report(y_test, y_pred))

##### Original dataset

In [ ]:
gradient_boosting_model(original_teams, 9)

##### Feature Selection dataset

In [ ]:
gradient_boosting_model(feature_selection_result, 9)

##### Feature Engineering dataset

In [ ]:
gradient_boosting_model(feature_engineering_result, 9)

In [ ]:
gradient_boosting_model(feature_engineering_result2, 9)

## Model Evaluation

### Decision Tree

In [ ]:
for i in range(1, 9): 
    print("Training year 9 based on previous ", i, " years.")
    filtered_feature_engineering_result2 = feature_engineering_result2[feature_engineering_result2['year'] >= 9 - i]
    decision_tree_model(filtered_feature_engineering_result2, 9)

### Random Forest

In [ ]:
for i in range(1, 9): 
    print("Training year 9 based on previous ", i, " years.")
    filtered_feature_engineering_result2 = feature_engineering_result2[feature_engineering_result2['year'] >= 9 - i]
    random_forest_model(filtered_feature_engineering_result2, 9)

### Logistic Regression

In [ ]:
for i in range(1, 9): 
    print("Training year 9 based on previous ", i, " years.")
    filtered_feature_engineering_result2 = feature_engineering_result2[feature_engineering_result2['year'] >= 9 - i]
    logistic_regression_model(filtered_feature_engineering_result2, 9)

### SVM

In [ ]:
for i in range(1, 9): 
    print("Training year 9 based on previous ", i, " years.")
    filtered_feature_engineering_result2 = feature_engineering_result2[feature_engineering_result2['year'] >= 9 - i]
    svm_model(filtered_feature_engineering_result2, 9)

### KNN

In [ ]:
for i in range(1, 9): 
    print("Training year 9 based on previous ", i, " years.")
    filtered_feature_engineering_result2 = feature_engineering_result2[feature_engineering_result2['year'] >= 9 - i]
    knn_model(filtered_feature_engineering_result2, 9)

### Gradient Boosting

In [ ]:
for i in range(1, 9): 
    print("Training year 9 based on previous ", i, " years.")
    filtered_feature_engineering_result2 = feature_engineering_result2[feature_engineering_result2['year'] >= 9 - i]
    gradient_boosting_model(filtered_feature_engineering_result2, 9)

## Testing for Final Dataset (Year 11)

In [ ]:
def final_decision_tree(dataset_train, dataset_predict):
    X = dataset_train.copy()
    y = dataset_train['playoff'].copy()
    X.drop(columns=['playoff'], inplace=True)

    X_train = dataset_train.drop(columns=['playoff'])
    y_train = dataset_train['playoff']

    X_test = dataset_predict
    #y_test = dataset[dataset['year'] == year]['playoff']

    # Define the hyperparameters and their possible values
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 2, 3, 4, 5, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Create a GridSearchCV object
    dt_classifier = DecisionTreeClassifier()
    grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Step 2: Extract the best parameters from Grid Search
    best_params = grid_search.best_params_

    # Step 3: Create a Decision Tree classifier with the best parameters
    best_model = DecisionTreeClassifier(**best_params)

    # Step 4: Perform Recursive Feature Elimination (RFE)
    rfe = RFE(estimator=best_model, n_features_to_select=1)
    X_rfe = rfe.fit_transform(X_train, y_train)

    # Step 5: Train the final Decision Tree model with the best parameters and selected features
    best_model.fit(X_rfe, y_train)

    # Step 6: Evaluate the model on the test set
    X_test_rfe = rfe.transform(X_test)
    y_pred = best_model.predict(X_test_rfe)

    # Display results
    results_df = pd.DataFrame({'Team ID': X_test['tmID'], 'Predicted Playoffs': y_pred})
    print(results_df)

In [ ]:
selected_features = feature_engineering_result2.columns.to_list()
selected_features.remove('playoff')
selected_features.remove('coachesPowerRanking')

feature_engineering_dataset = pd.read_csv('filtered/feature_engineering_dataset.csv', delimiter=",")
feature_engineering_dataset.drop(columns=['playoff', 'coachesPowerRanking'], inplace=True)

testing_data = pd.read_csv('filtered/team2_before_shift.csv', delimiter=",")
testing_data.drop(columns=['playoff'], inplace=True)
testing_data = pd.merge(testing_data, feature_engineering_dataset, on=['tmID', 'year'])
testing_data = testing_data[selected_features]
testing_data = testing_data[testing_data['year'] == 10]

testing_data.to_csv('filtered/testing_data.csv', index=False)

training_data = feature_engineering_result2.copy().drop(columns=['coachesPowerRanking'])

final_decision_tree(training_data, testing_data)